# Optimizing SQL and Python Pipelines for Data Science
> Poorly written SQL and Python can make data extraction and manipulation tedious and painful. Streamlined processes utilizing SQL best practices will save hours of frustration.

- toc: true 
- badges: true
- comments: true
- categories: [python, pandas, sql, data-science]
- permalink: /optpysql/

## Review SQL Best Practices

> Why the FROM statement is so important

- The FROM statement determines how you will structure the rest of your query
- You want your FROM table to be a core table with the fewest columns & rows that is highly indexed

```sql
SELECT users.*                              SELECT users.*
FROM users usr                              FROM orders or
INNER JOIN orders or                        INNER JOIN users usr
ON usr.customer_id = or.customer_id         ON usr.customer_id = or.customer_id
WHERE or.order_date >                       WHERE or.order_date >
date_sub(current_date, INTERVAL 30 days)    date_sub(current_date, INTERVAL 30 days)
```

**13 seconds to run** ------------------------------ **19 seconds to run**

Two queries that return exact same results. The only difference is we flip the tables, and it makes 6 seconds speed-up.

### Why Temporary Tables Rock

- Temporary tables improve readability over (nested) subqueries, i.e.

```sql
SELECT ...
INTO #goodnameforatemptable

SELECT ... 
FROM blah
JOIN #goodnameforatemptable

instead of 
SELECT ...
FROM 
JOIN (SELECT ... ) ON
```

- Keeps your code readable and makes troubleshooting much easier.
- Helps you follow the single responsibility principle.
- The [Query Optimizer](https://en.wikipedia.org/wiki/Query_optimization) may not be able to properly optimize a query with subqueries and will likely result in longer run times.


### Some Simple SQL Optimization things to remember

- When doing wildcard searches use `something%` (limit it to backend wildcard search) vs. `%something%` if you can.
- Functions on indexed columns in the where clause remove the indexing.

```sql
WHERE substring (column, 1, 1) = 'F'
```

vs.

```sql
WHERE column LIKE 'F%'
```

- Don't pull in columns you don't need.
- Move filters from the `WHERE` statement to the `JOIN` condition if using an `OUTER JOIN`.
- Use your indices as much as possible.
- If you can, use `UNION ALL` instead of `UNION DISTINCT`.

## Data Manipulation & Feature Engineering Before Python

### Joining Multiple Data Sources

- This is SQL's bread and butter, it's wheelhouse, what it was made to do.
- Even if you have multiple raw data files, I advocate for standing up a quick DB and loading the CSVs in there vs. bringing them straight into pandas &mdash; you don't have to reload the file every time you want to look at the data.

### Narrowing down your dataset

- More data isn't always better &mdash; loading 50 million rows of data straight into memory will be incredibly time consuming, inefficient, and does not necessarily lead to better machine learning scores.
- Time frame considerations &mdash; do you see a dramatic improvement in model scores with two years of data vs. one year?

### Learning Curves to Help Determine Dataset Sizes

Learning Curves help you understand not only your algorithm's [bias vs. variance](https://en.wikipedia.org/wiki/Bias%E2%80%93variance_tradeoff) but also how many records you need to train a model you're happy with.

- **High variance** &mdash; If a learning algorithm is suffering from high variance, getting more training data is likely to help.

- **High bias** &mdash; If a learning algorithm is suffering from high bias, getting more training data will not (by itself) help much.

### Feature Engineering

- Case statements prevent a size mismatch that can sometimes happen with `pandas.get_dummies()` in train vs test sets.
- Lead, Lag, Rank functions are powerful calculations that are oddly written in SQL but can leverage indices and run more optimally in SQL.
- A lot of features can be created before you even load the data if you'd like.

### Pandas

- You have the whole Python toolset at your disposal.
- You can do more advanced and intensive data manipulation in code that is easily readable and **testable**.
- Easy integration with data visualization libraries, jupyter notebooks, and functions like `data_frame.describe()` make it ideal for [EDA](https://en.wikipedia.org/wiki/Exploratory_data_analysis) (exploratory data analysis).

## How to Optimize Reading and Writing with Python

### Reading in data from SQL

- `pandas.read_sql()` and `pandas.read_gbq()` are notoriously slow.
- The best way I have found to do this is to save your final query as a table and export that table to CSV or CSV.GZ and then load the CSV into pandas via `pandas.read_csv()`
- What this also allows for is then each time moving forward you only have to load the CSV directly.

### Writing data to SQL

- Similarly, `data_frame.to_sql()`, `data_frame.to_gbq()`, even Spark's `data_frame.write.jdbc()` are also slow.

> Batch writing to the rescue.

- SQLAlchemy & Pandas:


```python
Session = sessionmaker(bind=dest_db_con)
sess = Session()
sess.bulk_insert_mappings(MentorInformation, df.to_dict(orient="records"))
sess.close()
```

- Spark:

```shell
jdbcUrl = 'jdbc:mysql://{}:3306/{}?useServerPrepStmts=false&rewriteBatchedStatements=true&user{}&password={}'
```

**&rewriteBatchedStatements=true**

With these steps, I've seen jobs that used to take hours now take minutes.

Conclusion:

- Knowing what tool is right for the job is extremely powerful.

Notes from:

- https://www.youtube.com/watch?v=H5FNFxHgSj8&list=LLqaZUbmWbk33CkdqqWfC1xw&index=2